# Seminario 5:  Python Mágico

[Problema a implementar](seminario_05_python_magico.md)
Implementación de la clase Matriz:

In [1]:
class Matriz:
    def __init__(self, filas, columnas, valor_inicial=0):
        self.filas = filas
        self.columnas = columnas
        self.matriz = [[valor_inicial]*columnas for _ in range(filas)]

    def __getitem__(self, indices):
        fila, columna = indices
        return self.matriz[fila][columna]

    def __setitem__(self, indices, valor):
        fila, columna = indices
        self.matriz[fila][columna] = valor
        
    def __getattr__(self, attr):
        fila, columna = map(int, attr.split('_')[1:])
        return self.matriz[fila][columna]

    def __setattr__(self, attr, valor):
        if attr.startswith('_'):
            fila, columna = map(int, attr.split('_')[1:])
            self.matriz[fila][columna] = valor
        else:
            # super().__setattr__(attr, valor)
            object.__setattr__(self, attr, valor)

    def __iter__(self):
        for fila in self.matriz:
            for valor in fila:
                yield valor

    def as_type(self, tipo):
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[tipo(valor) for valor in fila] for fila in self.matriz]
        return nueva_matriz

    def __str__(self):
        return '\n'.join([' '.join(map(str, fila)) for fila in self.matriz])

    # def _1_1 (self):
    #     print("se vuelve loco?")

    def __add__(self, other):
        if self.filas != other.filas or self.columnas != other.columnas:
            raise ValueError('Las matrices deben tener las mismas dimensiones')
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[self[fila, columna] + other[fila, columna] for columna in range(self.columnas)] for fila in range(self.filas)]
        return nueva_matriz
    
    def __sub__(self, other):
        if self.filas != other.filas or self.columnas != other.columnas:
            raise ValueError('Las matrices deben tener las mismas dimensiones')
        nueva_matriz = Matriz(self.filas, self.columnas)
        nueva_matriz.matriz = [[self[fila, columna] - other[fila, columna] for columna in range(self.columnas)] for fila in range(self.filas)]
        return nueva_matriz
    
    def __mul__(self, other):
        if self.columnas != other.filas:
            raise ValueError('El número de columnas de la primera matriz debe ser igual al número de filas de la segunda matriz')
        nueva_matriz = Matriz(self.filas, other.columnas)
        nueva_matriz.matriz = [[sum(self[fila, k] * other[k, columna] for k in range(self.columnas)) for columna in range(other.columnas)] for fila in range(self.filas)]
        return nueva_matriz


1. Implemente la clase `Matriz`, para representar matrices con las operaciones de suma y producto. Implemente además otras funcionalidades que crea necesarias.

In [6]:
matrizA = Matriz(2, 2, 1)
matrizB = Matriz(2, 2, 2)
print(f"MatrizA: \n{matrizA}\n")
print(f"MatrizB: \n{matrizB}\n")
print(f"A+B: \n{matrizA + matrizB}\n")
print(f"A-B: \n{matrizA - matrizB}\n")
print(f"A*B:\n{matrizA * matrizB}\n")

MatrizA: 
1 1
1 1

MatrizB: 
2 2
2 2

A+B: 
3 3
3 3

A-B: 
-1 -1
-1 -1

A*B:
4 4
4 4



2. Implemente la indización para la clase `Matriz` de forma tal que se puedan hacer construcciones como las siguientes: `a = matriz[0, 6]` o `matriz[1, 2] = 9`.

In [11]:
matrizA[1,1] = 10
print(f"A[1,1]: {matrizA[1,1]}")

a = matrizA[0, 0]
print(f"A[0,0]: {a}")

A[1,1]: 10
A[0,0]: 1


3. Implemente la indización para la clase `Matriz` por medio de acceso a campos de la forma: `a = matriz._0_6` o `matriz._1_2 = 9`.

In [12]:
matrizA._0_0 = 100
print(f"A[0,0]: {matrizA._0_0}")

a = matrizA._0_0
print(f"A[0,0]: {a}")

A[0,0]: 100
A[0,0]: 100


4. Los objetos matrices deberán ser iterables. El iterador de una matriz con `n` filas y `m` columnas debe devolver los elementos en el siguiente orden: `matriz_1_1, matriz_1_2, ..., matriz_1_m, matriz_2_1, ..., matriz_n_m`

In [15]:
print(f"MatrizA: \n{matrizA}\n")


a = iter(matrizA)

for valor in a:
    print(valor)

MatrizA: 
100 1
1 10

100
1
1
10


5. Al tipo matriz se podrá aplicar siempre el método `as_type()` que devuelve una nueva matriz con todos los tipos convertidos al tipo `type`. Suponga que existe un constructor en `type` que convierte de cualquier tipo a type. Por ejemplo:

``` c#
m = Matriz(2, 3) # crea una matriz de int con valor 0s.
mf = m.as_float() # mf es una matriz de 0s pero de tipo float.
```

In [24]:
m = Matriz(2, 3,5)
print(f"Matriz Original: \n{m}\n")
mf = m.as_type(float)
print(f"Matriz de float: \n{mf}\n")

Matriz Original: 
5 5 5
5 5 5

Matriz de float: 
5.0 5.0 5.0
5.0 5.0 5.0



##  Resolución de miembros y métodos en `Python`

// aqui viene todo lo del mro

## Métodos mágicos

Los métodos mágicos son métodos especiales que tienen doble guion bajo al principio y al final de sus nombres. Son métodos que nos permiten cambiar el comportamiento que van a tener nuestras clases o extender funcionalidades que ya existen en Python. Es un poco análogo a los operadores sobrecargados de C#.

Por qué los métodos especiales tienen una sintaxis rara?

"...he second bit of Python rationale I promised to explain is the reason
why I chose special methods to look __special__ and not merely
special. I was anticipating lots of operations that classes might want
to override, some standard (e.g. __add__ or __getitem__), some not so
standard (e.g. pickle's __reduce__ for a long time had no support in C
code at all). I didn't want these special operations to use ordinary
method names, because then pre-existing classes, or classes written by
users without an encyclopedic memory for all the special methods,
would be liable to accidentally define operations they didn't mean to
implement, with possibly disastrous consequences. Ivan Krstić
explained this more concise in his message, which arrived after I'd
written all this up..." - [Guido van Rossum](https://mail.python.org/pipermail/python-3000/2006-November/004643.html)

In [ ]:
Python 

Bibliografía:
https://mail.python.org/pipermail/python-3000/2006-November/004643.html
https://docs.python.org/3/faq/design.html 